# Data Preparation

## Setup

In [1]:
%%capture
%pip install kaggle

In [2]:
import pandas as pd

In [3]:
SAMPLE_NUM_CUSTOMERS = 1000 #set to 0 or less for no sampling
FILTER_ARTICLES = False #whether to filter out certain intimate products for demo purposes (real data problems)
RANDOM_SEED = 7474 #seed to use for replicating sampling

In [4]:
def camel_case(s):
    ss = s.lower().split('_')
    if len(ss) == 1:
        return ss[0]
    return ss[0] + ''.join(st.title() for st in ss[1:])

def camel_case_dict(name_keys):
    name_values = [camel_case(s) for s in name_keys]
    return dict(zip(name_keys, name_values))

def camel_case_rename_cols(df):
    col_map = camel_case_dict(df.columns)
    return df.rename(columns=col_map)

## Get Source Data

In [5]:
# configure authentication per instruction @ https://github.com/Kaggle/kaggle-api/blob/main/README.md
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f articles.csv -p data
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f customers.csv -p data
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f transactions_train.csv -p data
!cd data && unzip -n '*.zip'

articles.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
customers.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
transactions_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  customers.csv.zip

Archive:  articles.csv.zip

Archive:  transactions_train.csv.zip

3 archives were successfully processed.


## Filter Articles

In [6]:
init_article_df = camel_case_rename_cols(pd.read_csv('data/articles.csv'))
init_article_df.shape

(105542, 25)

In [7]:
# Filtering out some intimate products for demo purposes
filtered_article_ids = init_article_df.articleId
if FILTER_ARTICLES:
    filtered_article_ids = init_article_df[init_article_df.garmentGroupName != 'Under-, Nightwear'].articleId

In [8]:
init_transaction_df = camel_case_rename_cols(pd.read_csv('data/transactions_train.csv'))
# generally a good idea to have an id for these (source data doesn't include ids)
init_transaction_df['txId'] = range(init_transaction_df.shape[0])
init_transaction_df.shape

(31788324, 6)

In [9]:
filtered_customer_ids = init_transaction_df[init_transaction_df.articleId.isin(filtered_article_ids)].customerId.drop_duplicates()
filtered_customer_ids

0           000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
2           00007d2de826758b65a93dd24ce629ed66842531df6699...
7           00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...
12          0008968c0d451dbc5a9968da03196fe20051965edde741...
14          000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...
                                  ...                        
31788165    fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...
31788202    fecc5f77b5f7ee4570efde9ab05ec94d0de2bf80efb4f6...
31788208    fece2f68864c311a0b5208e2eb735b3dcde7e41461d327...
31788217    fee56cc5315dafb35a4490ccc6f711092cae913550c832...
31788275    ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...
Name: customerId, Length: 1362281, dtype: object

## Sample Customers

In [10]:
customer_ids = filtered_customer_ids
if SAMPLE_NUM_CUSTOMERS > 0:
    customer_ids = filtered_customer_ids.sample(n=SAMPLE_NUM_CUSTOMERS, random_state=RANDOM_SEED).reset_index(drop=True)
customer_ids

0      fdbe75e71e134938025dbbb9bc495bd302d578b449ac96...
1      fb9310441b653525f1adad3fbe7ece522ba50e752cca62...
2      f0a8599239eea199f1440af86ab9df78cb5d4e85f532fd...
3      4dd8a1b3175c88f07b123b388a5c9b5dfe16b3ba6fdf62...
4      696093ad8815f16ab92c07eb32d69c2d1e90daef479de7...
                             ...                        
995    8e43348085930e2063c673ce5f85579fcdb26293aa4efe...
996    8b555d0d3c601c3e2c734f8ef99810248d7532cf0cbee0...
997    83e43a73575426a3171610f218aeb638da207e6b73f869...
998    181c9e80cdddcfa2eb0436524bb6de520ee44dee99b527...
999    9f1dc11db29b3f458baeffb25e0644612b6aafa26c0728...
Name: customerId, Length: 1000, dtype: object

In [11]:
article_ids = init_transaction_df[init_transaction_df.customerId.isin(customer_ids)].articleId.drop_duplicates()
article_ids

2445        653428002
6182        636587001
6183        640462002
6188        645422002
6190        645424001
              ...    
31779121    906639004
31779122    684238003
31779124    921266007
31779125    812530004
31779126    942187001
Name: articleId, Length: 13351, dtype: int64

## Sample Down Data
Now that we have the list of customers and articles to include , `customer_ids`, and `article_ids` respectively, we can use them to filter the source data and stage for loading

In [12]:
transaction_df = init_transaction_df[init_transaction_df.customerId.isin(customer_ids)]
transaction_df

,tDat,customerId,articleId,price,salesChannelId,txId
2445,2018-09-20,0ddcd6055c5830c1fda493843d051edb04ce1bf888aa4b...,653428002,0.135576,1,2445
6182,2018-09-20,210f113fe87db5d6391e986dc06b8e4369e46284e3b989...,636587001,0.008458,1,6182
6183,2018-09-20,210f113fe87db5d6391e986dc06b8e4369e46284e3b989...,640462002,0.032186,1,6183
6188,2018-09-20,211a2ef477fcfc8fc40a63ffa70bb41086dd06ca85d4af...,645422002,0.014390,2,6188
6189,2018-09-20,211a2ef477fcfc8fc40a63ffa70bb41086dd06ca85d4af...,645422002,0.014390,2,6189
...,...,...,...,...,...,...
31779124,2020-09-22,b6be55f233772b5fc4a1ebedf36542fb3e1b6c15c23c7e...,921266007,0.016932,2,31779124
31779125,2020-09-22,b6be55f233772b5fc4a1ebedf36542fb3e1b6c15c23c7e...,812530004,0.010153,2,31779125
31779126,2020-09-22,b6be55f233772b5fc4a1ebedf36542fb3e1b6c15c23c7e...,942187001,0.016932,2,31779126
31779127,2020-09-22,b6be55f233772b5fc4a1ebedf36542fb3e1b6c15c23c7e...,866731001,0.025407,2,31779127


In [13]:
full_article_df = init_article_df[init_article_df.articleId.isin(article_ids)]
full_article_df

,articleId,productCode,prodName,productTypeNo,productTypeName,productGroupName,graphicalAppearanceNo,graphicalAppearanceName,colourGroupCode,colourGroupName,...,departmentName,indexCode,indexName,indexGroupNo,indexGroupName,sectionNo,sectionName,garmentGroupNo,garmentGroupName,detailDesc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
6,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
8,111586001,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with built-in support to lift the botto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105314,936862001,936862,EDC Marla dress,265,Dress,Garment Full body,1010001,All over pattern,52,Pink,...,Campaigns,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1023,Special Offers,Calf-length dress in a patterned Tencel™ lyoce...
105319,936979001,936979,Class Filippa Necklace,77,Necklace,Accessories,1010016,Solid,5,Gold,...,Jewellery,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories,Metal chain necklace with a pendant. Adjustabl...
105327,937138001,937138,Flirty Albin bracelet pk,68,Bracelet,Accessories,1010016,Solid,5,Gold,...,Jewellery Extended,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories,Metal chain bracelets. Two plain and two with ...
105444,942187001,942187,ED Sasha tee,255,T-shirt,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,"Oversized, straight-cut T-shirt in a soft moda..."


## Create Product, Department, and Article Dataframes

In [14]:
product_df = full_article_df[['productCode', 'prodName',
                              'productTypeNo', 'productTypeName',
                              'productGroupName', 'garmentGroupNo', 'garmentGroupName',
                              'detailDesc']].drop_duplicates(subset='productCode')

product_df.to_csv('product.csv', index=False)
product_df

,productCode,prodName,productTypeNo,productTypeName,productGroupName,garmentGroupNo,garmentGroupName,detailDesc
0,108775,Strap top,253,Vest top,Garment Upper body,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065,OP T-shirt (Idro),306,Bra,Underwear,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
6,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
8,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1021,Socks and Tights,Tights with built-in support to lift the botto...
9,111593,Support 40 den 1p Tights,304,Underwear Tights,Socks & Tights,1021,Socks and Tights,"Semi shiny tights that shape the tummy, thighs..."
...,...,...,...,...,...,...,...,...
105314,936862,EDC Marla dress,265,Dress,Garment Full body,1023,Special Offers,Calf-length dress in a patterned Tencel™ lyoce...
105319,936979,Class Filippa Necklace,77,Necklace,Accessories,1019,Accessories,Metal chain necklace with a pendant. Adjustabl...
105327,937138,Flirty Albin bracelet pk,68,Bracelet,Accessories,1019,Accessories,Metal chain bracelets. Two plain and two with ...
105444,942187,ED Sasha tee,255,T-shirt,Garment Upper body,1005,Jersey Fancy,"Oversized, straight-cut T-shirt in a soft moda..."


In [15]:
department_df = full_article_df[['departmentNo', 'departmentName', 'sectionNo', 'sectionName']]\
    .drop_duplicates(subset='departmentNo')

department_df.to_csv('department.csv', index=False)
department_df

,departmentNo,departmentName,sectionNo,sectionName
0,1676,Jersey Basic,16,Womens Everyday Basics
3,1339,Clean Lingerie,61,Womens Lingerie
6,3608,Tights basic,62,"Womens Nightwear, Socks & Tigh"
17,5883,Jersey Basic,26,Men Underwear
23,2032,Jersey,8,Mama
...,...,...,...,...
92964,7510,Woven,28,Men Edition
97621,3420,Small Accessories Extended,66,Womens Small accessories
98705,5231,Jacket,31,Mens Outerwear
101971,8090,Promotion/Other/Offer,29,Men Other


In [16]:
article_df = full_article_df[['articleId', 'productCode', 'departmentNo', 'prodName', 'productTypeName',
                              'graphicalAppearanceNo', 'graphicalAppearanceName', 'colourGroupCode', 'colourGroupName']]
article_df.to_csv('article.csv', index=False)
article_df

,articleId,productCode,departmentNo,prodName,productTypeName,graphicalAppearanceNo,graphicalAppearanceName,colourGroupCode,colourGroupName
0,108775015,108775,1676,Strap top,Vest top,1010016,Solid,9,Black
1,108775044,108775,1676,Strap top,Vest top,1010016,Solid,10,White
3,110065001,110065,1339,OP T-shirt (Idro),Bra,1010016,Solid,9,Black
6,111565001,111565,3608,20 den 1p Stockings,Underwear Tights,1010016,Solid,9,Black
8,111586001,111586,3608,Shape Up 30 den 1p Tights,Leggings/Tights,1010016,Solid,9,Black
...,...,...,...,...,...,...,...,...,...
105314,936862001,936862,3090,EDC Marla dress,Dress,1010001,All over pattern,52,Pink
105319,936979001,936979,4344,Class Filippa Necklace,Necklace,1010016,Solid,5,Gold
105327,937138001,937138,4345,Flirty Albin bracelet pk,Bracelet,1010016,Solid,5,Gold
105444,942187001,942187,1919,ED Sasha tee,T-shirt,1010016,Solid,9,Black


In [17]:
customer_df = camel_case_rename_cols(pd.read_csv('data/customers.csv'))
customer_df = customer_df[customer_df.customerId.isin(customer_ids)]
customer_df.to_csv('customer.csv', index=False)
customer_df

,customerId,fn,active,clubMemberStatus,fashionNewsFrequency,age,postalCode
810,00264b7d4cd6498292e8a355b699c2d07725d123f04867...,1.0,1.0,ACTIVE,Regularly,53.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
1947,005c6d3bb66c86aab606814cd9995a12f99b3a44b58c72...,NaN,NaN,PRE-CREATE,NONE,NaN,177b4a2258a85a2247daaa7cdffba96a74c741ea8a6605...
3548,00abec3de294e03d192db15b91e154853ee1c89415e7cd...,NaN,NaN,ACTIVE,NONE,49.0,86557a458110ac98f4ca80e5a815ba2e8ea086dd8039b0...
5027,00f311a42124fc44d117135f34e1fca29fcac271e6fbd0...,1.0,1.0,ACTIVE,Regularly,55.0,1a80c5651ae36327a86e71d5b967cf62c31126d1b57ae0...
6432,0132cd2eb3c6b1f66784f65f94ddd8352add2653e0caf5...,NaN,NaN,ACTIVE,NONE,49.0,49f7ec29bcacbbf2120af5162f9f99c212e9dd26b48d79...
...,...,...,...,...,...,...,...
1361059,fdf1294f414faac2b00a725f5d80c34f98a744d9b8b3ce...,NaN,NaN,ACTIVE,NONE,32.0,0cd87888c3a13ebbb1e90cac6b9fbf34c51afa40865f55...
1363646,fe6faeed37fe86e885928d3ab30d8d9b072d6643c8aa15...,1.0,1.0,ACTIVE,Regularly,46.0,fe234b03107b233aec5695dc4c3fbe8e638338643f4e14...
1366543,fef793ec3a7d62d782824517355d74ded50964dce33009...,NaN,NaN,ACTIVE,NONE,46.0,5799a39cffe701ebdb12181348bf10f9e23abcc3868c43...
1370498,ffb925b11e1bb2e375d22a02d67907994eb8cb92ec2e7d...,NaN,NaN,ACTIVE,NONE,34.0,ebdd8c5c893683c3cf52c011d4e35024e46d183c95f0fa...


## Write Transactions Dataframe to csv

In [18]:
transaction_df.to_csv('transaction.csv', index=False)

## Upload to Google Cloud

In [19]:
!gsutil cp product.csv gs://neo4j-workshop-data/genai-hm

Copying file://product.csv [Content-Type=text/csv]...
/ [1 files][  1.6 MiB/  1.6 MiB]                                                
Operation completed over 1 objects/1.6 MiB.                                      


In [20]:
!gsutil cp department.csv gs://neo4j-workshop-data/genai-hm

Copying file://department.csv [Content-Type=text/csv]...
/ [1 files][ 10.4 KiB/ 10.4 KiB]                                                
Operation completed over 1 objects/10.4 KiB.                                     


In [21]:
!gsutil cp article.csv gs://neo4j-workshop-data/genai-hm

Copying file://article.csv [Content-Type=text/csv]...
- [1 files][959.7 KiB/959.7 KiB]                                                
Operation completed over 1 objects/959.7 KiB.                                    


In [22]:
!gsutil cp customer.csv gs://neo4j-workshop-data/genai-hm

Copying file://customer.csv [Content-Type=text/csv]...
/ [1 files][149.5 KiB/149.5 KiB]                                                
Operation completed over 1 objects/149.5 KiB.                                    


In [23]:
!gsutil cp transaction.csv gs://neo4j-workshop-data/genai-hm

Copying file://transaction.csv [Content-Type=text/csv]...
- [1 files][  2.6 MiB/  2.6 MiB]                                                
Operation completed over 1 objects/2.6 MiB.                                      
